# Capstone 3 Data Wrangling- Jessica Williams

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from yellowbrick.text import FreqDistVisualizer
from pathlib import Path
nlp=spacy.load('en_core_web_sm')
from textblob import TextBlob, Word, Blobber

from sb_utils import save_file

Lets take a look at both dataframes.

In [2]:
wine_list = pd.read_csv('C:/Users/jwatki8/Downloads/XWines_Test_100_wines.csv')

In [3]:
wine_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WineID      100 non-null    int64  
 1   WineName    100 non-null    object 
 2   Type        100 non-null    object 
 3   Elaborate   100 non-null    object 
 4   Grapes      100 non-null    object 
 5   Harmonize   100 non-null    object 
 6   ABV         100 non-null    float64
 7   Body        100 non-null    object 
 8   Acidity     100 non-null    object 
 9   Code        100 non-null    object 
 10  Country     100 non-null    object 
 11  RegionID    100 non-null    int64  
 12  RegionName  100 non-null    object 
 13  WineryID    100 non-null    int64  
 14  WineryName  100 non-null    object 
 15  Website     100 non-null    object 
 16  Vintages    100 non-null    object 
dtypes: float64(1), int64(3), object(13)
memory usage: 13.4+ KB


In [4]:
wine_list.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100062,Origem Merlot,Red,Varietal/100%,['Merlot'],"['Beef', 'Lamb', 'Veal', 'Grilled', 'Pizza', '...",13.0,Full-bodied,Medium,BR,Brazil,1002,Vale dos Vinhedos,10014,Casa Valduga,http://www.casavalduga.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100191,Reserva Chardonnay,White,Varietal/100%,['Chardonnay'],"['Rich Fish', 'Seafood', 'Risotto', 'Poultry',...",13.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
2,101847,Dona Antonia Porto Reserva Tawny,Dessert/Port,Assemblage/Blend,"['Touriga Nacional', 'Touriga Franca', 'Tinta ...","['Appetizer', 'Sweet Dessert', 'Blue Cheese']",20.0,Very full-bodied,High,PT,Portugal,1031,Porto,10674,Porto Ferreira,https://sogrape.com/pt/brand/porto-ferreira,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,102055,Fine Ruby Port,Dessert/Port,Assemblage/Blend,"['Tinta Amarela', 'Tinta Barroca', 'Touriga Fr...","['Sweet Dessert', 'Cake', 'Fruit', 'Soft Cheese']",19.5,Very full-bodied,Medium,PT,Portugal,1031,Porto,10703,Sandeman,http://www.sandeman.com,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
4,102079,Maré Alta,White,Assemblage/Blend,"['Loureiro', 'Alvarinho', 'Arinto']","['Fish', 'Shellfish', 'Vegetarian', 'Appetizer...",10.0,Very light-bodied,High,PT,Portugal,1034,Vinho Verde,11486,Fonte Pequena,https://www.casadafontepequena.pt,"[2022, 2021, 2020, 2019, 2018, 2017, 2016, 201..."


In [5]:
wine_ratings = pd.read_csv('C:/Users/jwatki8/Downloads/XWines_Test_1K_ratings.csv')

In [6]:
wine_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   RatingID  1000 non-null   int64  
 1   UserID    1000 non-null   int64  
 2   WineID    1000 non-null   int64  
 3   Vintage   1000 non-null   object 
 4   Rating    1000 non-null   float64
 5   Date      1000 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 47.0+ KB


In [7]:
wine_ratings.head()

,RatingID,UserID,WineID,Vintage,Rating,Date
0,3211,1209683,111478,1959,4.5,2016-08-08 00:50:22
1,27878,1209980,111478,1975,4.0,2018-08-12 17:09:39
2,31227,1258705,111478,1975,5.0,2014-11-16 19:52:38
3,41946,1139706,111478,1979,5.0,2014-12-22 02:30:15
4,61700,1240747,111478,1982,4.5,2019-10-21 02:01:10


In [8]:
wine_list.shape

(100, 17)

In [9]:
wine_list.dtypes

WineID          int64
WineName       object
Type           object
Elaborate      object
Grapes         object
Harmonize      object
ABV           float64
Body           object
Acidity        object
Code           object
Country        object
RegionID        int64
RegionName     object
WineryID        int64
WineryName     object
Website        object
Vintages       object
dtype: object

The first thing I would like to do with the wine list dataframe is drop the website column as I don’t see this as being of any use.

In [10]:
wine_list=wine_list.drop(columns='Website')

In [11]:
wine_list.dtypes

WineID          int64
WineName       object
Type           object
Elaborate      object
Grapes         object
Harmonize      object
ABV           float64
Body           object
Acidity        object
Code           object
Country        object
RegionID        int64
RegionName     object
WineryID        int64
WineryName     object
Vintages       object
dtype: object

Let’s check for null values

In [12]:
wine_list.isnull().sum()

WineID        0
WineName      0
Type          0
Elaborate     0
Grapes        0
Harmonize     0
ABV           0
Body          0
Acidity       0
Code          0
Country       0
RegionID      0
RegionName    0
WineryID      0
WineryName    0
Vintages      0
dtype: int64

Now I want to examine the unique values in the different string attribute columns.

In [13]:
wine_list['Type'].value_counts()

Red             53
White           19
Rosé             8
Dessert          8
Sparkling        7
Dessert/Port     5
Name: Type, dtype: int64

In [14]:
wine_list['Elaborate'].value_counts()

Varietal/100%                        58
Assemblage/Blend                     25
Varietal/>75%                         5
Assemblage/Bordeaux Red Blend         4
Assemblage/Portuguese Red Blend       2
Assemblage/Provence Rosé Blend        2
Assemblage/Valpolicella Red Blend     2
Assemblage/Port Blend                 1
Assemblage/Rhône Red Blend            1
Name: Elaborate, dtype: int64

In [15]:
wine_list['Grapes'].value_counts()

['Pinot Noir']                                      13
['Chardonnay']                                       7
['Nebbiolo']                                         5
['Riesling']                                         3
['Cabernet Franc']                                   3
                                                    ..
['Pinot Grigio', 'Chardonnay']                       1
['Cabernet Sauvignon', 'Merlot', 'Petit Verdot']     1
['Negroamaro', 'Merlot', 'Refosco']                  1
['Müller-Thurgau']                                   1
['Pinot Gris']                                       1
Name: Grapes, Length: 65, dtype: int64

In [16]:
wine_list['Harmonize'].value_counts()

['Beef', 'Veal', 'Game Meat', 'Poultry']                                  9
['Beef', 'Lamb', 'Game Meat', 'Poultry']                                  6
['Beef', 'Pasta', 'Lamb', 'Game Meat']                                    5
['Beef', 'Lamb', 'Poultry']                                               3
['Pasta', 'Veal', 'Pork', 'Game Meat', 'Poultry']                         2
                                                                         ..
['Pork', 'Rich Fish', 'Shellfish', 'Soft Cheese']                         1
['Pork', 'Rich Fish', 'Spicy Food', 'Vegetarian', 'Cured Meat']           1
['Appetizer', 'Snack', 'Pasta', 'Chicken', 'Vegetarian']                  1
['Rich Fish', 'Fruit Dessert', 'Vegetarian', 'Poultry', 'Soft Cheese']    1
['Pork', 'Spicy Food', 'Mushrooms', 'Vegetarian']                         1
Name: Harmonize, Length: 76, dtype: int64

In [17]:
wine_list['Body'].value_counts()

Medium-bodied        46
Full-bodied          32
Very full-bodied     12
Light-bodied          9
Very light-bodied     1
Name: Body, dtype: int64

In [18]:
wine_list['Acidity'].value_counts()

Medium    54
High      45
Low        1
Name: Acidity, dtype: int64

Next let’s check the wine Id variable for duplicates.

In [19]:
wine_list['WineID'].value_counts()

100062    1
160271    1
168457    1
168182    1
167464    1
         ..
120737    1
119803    1
118927    1
118457    1
198580    1
Name: WineID, Length: 100, dtype: int64

In [20]:
wine_list['WineID'].value_counts().max()

1

It looks like the ids for the recipes are all unique.

Let’s examine the wine rating dataframe a little closer.

In [21]:
wine_ratings.dtypes

RatingID      int64
UserID        int64
WineID        int64
Vintage      object
Rating      float64
Date         object
dtype: object

Once again we will check for null values.

In [22]:
wine_ratings.isnull().sum()

RatingID    0
UserID      0
WineID      0
Vintage     0
Rating      0
Date        0
dtype: int64

In [23]:
wine_ratings['Vintage'].value_counts()

2015    98
2014    92
2016    83
2013    83
2012    81
2017    77
2018    69
2011    64
2008    52
2010    47
N.V.    43
2020    41
2019    34
2009    26
2004    16
2000    14
2007    13
2006    11
1989    10
2005     8
1999     7
2003     4
1998     4
2001     3
1982     3
1988     3
1975     2
2021     2
2002     2
1983     1
1979     1
1996     1
1987     1
1993     1
1994     1
1997     1
1959     1
Name: Vintage, dtype: int64

Now let’s take a look at the value range for the ratings columns in this dataframe.

In [24]:
print(wine_ratings['Rating'].min())
print(wine_ratings['Rating'].max())

1.0
5.0


In [25]:
wine_ratings['WineID'].value_counts()

102055    25
179386    24
167079    23
111478    22
112875    22
          ..
120737     5
143806     5
102645     5
175908     5
154979     5
Name: WineID, Length: 100, dtype: int64

I would like to get the mean of the rating column grouped by Wine ID. In other words find the mean of the ratings for each recipe and then apply it to the recipe data frame by id#. Looking at the value counts for the Wine ID column in both data frames I can see that they contain the same number of unique values in each column. This mean that there is at least some rating data for all recipe ids. Now let’s calculate the mean rating.

In [26]:
wine_ratings['rating_mean'] = wine_ratings['Rating'].groupby(wine_ratings['WineID']).transform('mean')

In [27]:
wine_ratings.head()

,RatingID,UserID,WineID,Vintage,Rating,Date,rating_mean
0,3211,1209683,111478,1959,4.5,2016-08-08 00:50:22,4.136364
1,27878,1209980,111478,1975,4.0,2018-08-12 17:09:39,4.136364
2,31227,1258705,111478,1975,5.0,2014-11-16 19:52:38,4.136364
3,41946,1139706,111478,1979,5.0,2014-12-22 02:30:15,4.136364
4,61700,1240747,111478,1982,4.5,2019-10-21 02:01:10,4.136364


In [28]:
wine_ratings.tail()

,RatingID,UserID,WineID,Vintage,Rating,Date,rating_mean
995,20871491,1006657,102055,N.V.,3.5,2019-12-29 13:38:44,3.580000
996,20871989,1160496,160271,N.V.,4.0,2015-01-07 12:39:36,3.916667
997,20875704,1055576,102055,N.V.,3.5,2020-11-03 19:14:49,3.580000
998,20885463,1004369,102055,N.V.,3.5,2020-08-29 01:26:13,3.580000
999,20889646,1243678,102055,N.V.,3.5,2019-09-04 18:15:31,3.580000


Let’s look at the minimum and maximum values for our new rating mean column.

In [29]:
print(wine_ratings['rating_mean'].min())
print(wine_ratings['rating_mean'].max())


2.7142857142857144
4.875


Next let’s take a look at some of the user information. Let’s see how many unique raters we have out of 1000 ratings.

In [30]:
wine_ratings['UserID'].nunique()

636

Our dataset contains 636 unique raters. Let’s examine further.

In [31]:
print(wine_ratings['UserID'].value_counts().max())

5


In [32]:
wine_ratings['UserID'].value_counts().head(20)

1004940    5
1123010    5
1100007    5
1084433    5
1070726    4
1000064    4
1165941    4
1119279    4
1004804    4
1188640    4
1009797    4
1196127    4
1009381    4
1003868    4
1863413    4
1070932    4
1006657    3
1000045    3
1055113    3
1209683    3
Name: UserID, dtype: int64

In [33]:
wine_ratings['UserID'].value_counts().tail(20)

1687371    1
1145862    1
1267880    1
1222784    1
1165079    1
1219368    1
1189736    1
1003724    1
1176849    1
1169832    1
1324684    1
1456446    1
1086669    1
1634303    1
1979847    1
2022471    1
1434293    1
1209706    1
1844375    1
1221444    1
Name: UserID, dtype: int64

It looks like the maximum number of times someone reviewed a wine was 5. In examining the head and the tail of these values we can see that or max of users who rated 5 times didn’t occur very often. Only 4 of the 636 raters reviewed a wine at least 5 times.

In [35]:
datapath ='C:/Users/jwatki8/Documents/Data Science Program information/My Capstone project'
save_file(wine_list,'wine_list.csv', datapath)

Writing file.  "C:/Users/jwatki8/Documents/Data Science Program information/My Capstone project\wine_list.csv"
